In [ ]:
import numpy as np
import pandas as pd
import pymongo

In [ ]:
oscar_df = pd.read_csv('../../dataset/the_oscar_awards.csv')
oscar_df

In [ ]:
oscar_df.dtypes

In [ ]:
oscar_df['category'] = oscar_df['category'].astype(pd.StringDtype())
oscar_df['name'] = oscar_df['name'].astype(pd.StringDtype())
oscar_df['film'] = oscar_df['film'].astype(pd.StringDtype())

In [ ]:
oscar_df.dtypes

In [ ]:
oscar_df.loc[oscar_df.isna().any(axis=1)]

In [ ]:
#oscar_df = oscar_df.loc[~oscar_df.isna().any(axis=1)].copy()

In [ ]:
oscar_df.loc[oscar_df.duplicated(subset=['category','name','film']) & ~oscar_df['film'].isna()]

In [ ]:
oscar_df = oscar_df.loc[~oscar_df.duplicated(subset=['category','name','film']) & ~oscar_df['film'].isna()]

In [ ]:
oscar_df

In [ ]:
oscar_df = oscar_df.replace({np.nan: None})

In [ ]:
oscar_grouped = []
for movie , group in oscar_df.groupby('film'):
    oscars = group.apply(lambda row:{
        "year_film": row['year_film'],
        "year_ceremony": row['year_ceremony'],
        "ceremony": row['ceremony'],
        "category": row['category'],
        "name": row['name'],
        "film": row['film'],
        "winner": row['winner'],
    }, axis=1).tolist()

    oscar_grouped.append({
        "movie_title" : movie ,
        "oscars" : oscars
    })

In [ ]:
oscar_df.to_csv('../../dataset/cleaned/.csv', index=False)

In [ ]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["RT_reviews_oscars"]
mycol = mydb["Oscars"]
mycol.insert_many(oscar_grouped)

In [ ]:
# Conta quante categorie uniche ci sono nella colonna 'category'
num_unique_categories = oscar_df['category'].nunique()
print(f"Numero di categorie uniche: {num_unique_categories}")